<a href="https://colab.research.google.com/github/100477706/Proyecto_Aprendizaje/blob/main/modelo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MODELO FINAL**

En el notebook de predicciones entrenamos distintos modelos, ajustando sus hiperparámetros, y los evaluamos utilizando el método de validación cruzada (inner) para escoger el mejor de ellos. El modélo escogido termino siendo la SVM con los parámetros 'C' = 1.4, 'class_weight' = 'balanced' y 'kernel'= 'rbf', el cuál evaluamos con el método holdout (outer) para obtener la prédicción de cómo va a desepeñarse el mismo en el futuro y en la competición, este score fue de 85%. Ahora debemos pasar a entrenar el modelo con todos los datos disponibles y realizar nuestras predicciones para la competición.

#**CARGA DE DATOS**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, accuracy_score, recall_score
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score
from sklearn.compose import ColumnTransformer
from sklearn import tree
from sklearn import metrics
import numpy as np
import time

In [ ]:
df = pd.read_csv("attrition_availabledata_06.csv.gz", compression="gzip", sep=",") #subir el archivo comprimido
df.head()

#**PREPROCESO DE DATOS**

In [ ]:
# Columnas constantes y de ID
irrelevant_cols = ['Attrition', 'EmployeeID', 'EmployeeCount', 'StandardHours', 'Over18']
categorical_vars = [col for col in categorical_vars if col not in irrelevant_cols]
numerical_vars = [col for col in numerical_vars if col not in irrelevant_cols]

# Retiramos las columnas innecesarias y hacemos la división entre train y test
X = df.drop(columns=irrelevant_cols)
# Como la variable 'Attrition' es YES o NO, entonces la convertimos a un valor
# binario para el modelo
y = df['Attrition'].map({'Yes': 1, 'No': 0})

#**ENTRENO DEL MODELO**

In [ ]:
# Preprocesador para datos numéricos: imputación con la mediana + escalado robusto
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler())
])

# Preprocesador para datos categóricos: OneHotEncoding
cat_transformer = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar ambos en un ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_transformer, numerical_vars),
    ('cat', cat_transformer, categorical_vars)
])

# Pipeline final con preprocesamiento y el modelo final y los mejores parámetros
final_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=100477706, C=1.4, class_weight='balanced', kernel='rbf'))
])

# Entrenamos el modelo con los datos de train
final_model = final_pipe.fit(X, y)

#**PREDICCIONES DE COMPETICIÓN**